In [1]:
from dataclasses import dataclass, field
import time
from typing import Optional

from dataset.math_dataset import MathDatasetLoader
from utils import *
from utils.logger import *
from utils.utils import *
from utils.math import *
from utils.model import *
from utils.qwen_math_parser import *

logger = DefaultProgressLogger()

/data/scratch/junhong/miniconda3/envs/vllm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-16 00:21:58,149 | INFO | Task-4 | model:<module>:22 - Using Qwen/Qwen2.5-Math-PRM-7B for LLM response.


In [7]:
from datasets import load_dataset
dataset = load_dataset("UWNSL/MATH_training_split_long_cot", split="train")

Generating train split: 100%|█████████████████████████████████████████████| 5383/5383 [00:00<00:00, 14469.23 examples/s]


In [8]:
dataset

Dataset({
    features: ['problem', 'solution'],
    num_rows: 5383
})

In [ ]:
system_prompt='''Given a chain-of-thought reasoning process, your task is to identify and categorize each reasoning step into one of the following categories:

1. continue reasoning: encourages the model to build upon its current reasoning trajectory by generating the next intermediate step.
2. reflect: prompts the model to pause and verify the correctness of current reasoning path or calculation results. Marked by some keywords like "wait" and "verify".
3. explore: signals the model to explore a alternative and new solution. Marked by some keywords like "alternative" and "another approach".

Here is an example to follow:

## Instruction: Let $\mathcal{C}$ be the hyperbola $y^2 - x^2 = 1$. Given a point $P_0$ on the $x$-axis, we construct a sequence of points $(P_n)$ on the $x$-axis in the following manner: let $\ell_n$ be the line with slope $n$ passing through $P_n$, then $P_{n+1}$ is the orthogonal projection of the point of intersection of $\ell_n$ and $\mathcal C$ onto the $x$-axis. (If $P_n = 0$, then the sequence simply terminates.) Find the sum of the $x$-coordinates of all points $P_n$ if $P_0 = (1,0)$. Your answer should use only integers and $\ln$..

## Response: Given the hyperbola \(\mathcal{C}\) defined by the equation \(y^2 - x^2 = 1\), we start with the point \(P_0 = (1,0)\). The line \(\ell_0\) passing through \(P_0\) with slope \(0\) is the horizontal line \(y = 0\). The intersection of \(\ell_0\) with \(\mathcal{C}\) is the point \((1,0)\) and \((-1,0)\). The orthogonal projection of \((-1,0)\) onto the \(x\)-axis is \((-1,0)\), so \(P_1 = (-1,0)\).\n\nBut wait. The intersection of \(\ell_0\) with \(\mathcal{C}\) is not \((1,0)\) and \((-1,0)\); it is only \((0,1)\) and \((0,-1)\).\n\nAs an alternative, The correct intersection point to consider is \((0,1)\), and the orthogonal projection of \((0,1)\) onto the \(x\)-axis is \((0,0)\), so \(P_1 = (0,0)\).\n\nLet's re-evaluate the problem correctly. Given the hyperbola \(\mathcal{C}\) defined by the equation \(y^2 - x^2 = 1\), we start with the point \(P_0 = (1,0)\). The line \(\ell_0\) passing through \(P_0\) with slope \(n\) is given by \(y = n(x-1)\). We need to find the intersection of \(\ell_0\) with \(\mathcal{C}\).\n\nSubstitute \(y = n(x-1)\) into the hyperbola equation: \[ (n(x-1))^2 - x^2 = 1 \] \[ n^2(x^2 - 2x + 1) - x^2 = 1 \] \[ n^2x^2 - 2n^2x + n^2 - x^2 = 1 \] \[ (n^2 - 1)x^2 - 2n^2x + n^2 - 1 = 0 \] This is a quadratic equation in \(x\).\n\nUsing the quadratic formula \(x = \frac{-b \pm \sqrt{b^2 - 4ac}}{2a}\), where \(a = n^2 - 1\), \(b = -2n^2\), and \(c = n^2 - 1\), we get: \[ x = \frac{2n^2 \pm \sqrt{(2n^2)^2 - 4(n^2 - 1)(n^2 - 1)}}{2(n^2 - 1)} \] \[ x = \frac{2n^2 \pm \sqrt{4n^4 - 4(n^4 - 2n^2 + 1)}}{2(n^2 - 1)} \] \[ x = \frac{2n^2 \pm \sqrt{4n^4 - 4n^4 + 8n^2 - 4}}{2(n^2 - 1)} \] \[ x = \frac{2n^2 \pm \sqrt{8n^2 - 4}}{2(n^2 - 1)} \] \[ x = \frac{2n^2 \pm 2\sqrt{2n^2 - 1}}{2(n^2 - 1)} \] \[ x = \frac{n^2 \pm \sqrt{2n^2 - 1}}{n^2 - 1} \] The two solutions are \(x = \frac{n^2 + \sqrt{2n^2 - 1}}{n^2 - 1}\) and \(x = \frac{n^2 - \sqrt{2n^2 - 1}}{n^2 - 1}\). Since \(P_0 = (1,0)\) is on the right side of the hyperbola, we take the negative root to ensure the sequence converges to zero.\n\nThus, the \(x\)-coordinate of \(P_{n+1}\) is: \[ x_{n+1} = \frac{n^2 - \sqrt{2n^2 - 1}}{n^2 - 1} \] We can simplify this expression further: \[ x_{n+1} = 1 - \frac{\sqrt{2n^2 - 1}}{n^2 - 1} \] To find the sum of the \(x\)-coordinates of all points \(P_n\), we need to sum the series: \[ \sum_{n=0}^{\infty} x_n \] By examining the pattern, we see that each term \(x_n\) approaches 0 as \(n\) increases. The sum of the series can be found using properties of logarithms and hyperbolic functions. The result is: \[ \sum_{n=0}^{\infty} x_n = \boxed{1} \]

## Output: 
{{
    "Given the hyperbola ...": "continue reasoning",
    "But wait. The intersection of...": "reflect-effective",
    "As an alternative, The correct intersection...": "explore-0",
    "Let's re-evaluate the problem correctly...": "reflect-ineffective",
    "Substitute \(y = n(x-1)\) into...": "continue reasoning",
    "Using the quadratic formula...": "continue reasoning",
    "Thus, the \(x\)-coordinate of...": "continue reasoning"
}}

Noted that each reasoning step is seperated by \n\n. As the demonstration shows, each single reasoning step's brief content and its reasoning category should be a key-value pair in the dictionary. For the explore step, the number followed by "explore" means which step it goes back to and explores from. For example, explore-0 means the reasoning process goes back to the very beginning and tries to find an alternative solution. Only reflecting steps that lead to a different answer should be reflect-effective, otherwise, it should be reflect-ineffective.
Formatted the example above and think step by step to categorize reasoning step for the following case:'''

user_message = """
##Instruction: {problem}

##Response: {response}

##Output:"
"""

prompt = user_message.format(
    problem=problem, 
    response=response
)
generate_response()

In [ ]:
from typing import Union, List
from abc import ABC, abstractmethod
import math
from enum import Enum

class AggregationMethod(Enum):
    MODEL = "model"
    PRODUCT = "prod"
    MIN = "min"
    LAST = "last"

class AbstractProcessRewardModel(ABC):
    """abstract base class for process reward models"""

    @abstractmethod
    def score(self, question: str, responses: List[str], step_sep: str = "\n\n", aggregation_method: str = None, return_full_prm_result: bool = False, max_input_tokens: int = 8196) -> Union[List[PRMResult], List[float]]:
        """the reward for the given steps"""
        pass

import torch
from transformers import AutoTokenizer
from typing import Union, List
import os
from vllm import LLM

class VllmProcessRewardModel(AbstractProcessRewardModel):
    def __init__(self, model_name: str, device: Union[str, int], **kwargs):
        os.environ["VLLM_ALLOW_LONG_MAX_MODEL_LEN"] = "1"
        num_gpus = torch.cuda.device_count()
        print(f"Number of GPUs: {num_gpus}")
        
        self.model = LLM(model=model_name, 
                    task="reward",
                    device=device,
                    gpu_memory_utilization=0.8,
                    tensor_parallel_size=1,
                    )
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        if model_name == "Qwen/Qwen2.5-Math-PRM-7B":
            self.tokenizer.truncation_side = "left"
        self.model_name = model_name

    def score(self, messages: Union[List[List[dict]], List[dict]], max_input_tokens: int = 8192, step_sep: str = "\n\n",
             aggregation_method: Union[AggregationMethod, str] = AggregationMethod.LAST, return_full_prm_result: bool = False, use_tqdm: bool = False) -> Union[List[PRMResult], List[float]]:
        """
        Score last turn assistant message using the OpenAI chat completion format.
        
        Args:
            messages: List of conversations in OpenAI chat completion format
            max_input_tokens: Maximum number of input tokens
            return_full_prm_result: Whether to return full PRM results
            use_tqdm: Whether to display a progress bar (using tqdm)
        """
        if isinstance(aggregation_method, str):
            aggregation_method = AggregationMethod(aggregation_method)
        if isinstance(messages[0], dict):
            # ensure the input is a list of list of dicts   
            messages = [messages]
        if self.model_name == "Qwen/Qwen2.5-Math-PRM-7B":
            formatted_convs = []
            QWEN_PRM_SYSTEM_PROMPT = "Please reason step by step, and put your final answer within \\boxed{}."
            system_turn = [{
                "role": "system",
                "content": QWEN_PRM_SYSTEM_PROMPT
            }]
            for conv_messages in messages:

                last_assistant_message = conv_messages[-1]['content']
                if aggregation_method == AggregationMethod.MODEL:
                    steps_list = [last_assistant_message]
                else:
                    steps_list = last_assistant_message.split(step_sep)
                formatted_last_assistant_turn = [
                    {"role": "assistant", "content": "<extra_0>".join(steps_list) + "<extra_0>"}
                ]
                prepared_messages = system_turn + conv_messages[:-1] + formatted_last_assistant_turn
                conversation = self.tokenizer.apply_chat_template(
                    prepared_messages,
                    tokenize=False, 
                    add_generation_prompt=False
                )
                formatted_convs.append(conversation)

            all_input_ids = self.tokenizer(
                formatted_convs,
                return_tensors="pt", 
                truncation=True,
                padding=True,
                max_length=max_input_tokens
            ).input_ids
            batch_decoded = self.tokenizer.batch_decode(all_input_ids, skip_special_tokens=False)
            all_outputs = self.model.encode(batch_decoded, use_tqdm=use_tqdm)
            all_scores = [[d[-1].item() for d in ex.outputs.data] for ex in all_outputs]

        else:
            raise ValueError(f"Model {self.model_name} is not supported")
        
        if return_full_prm_result:
            return [PRMResult(scores=scores) for scores in all_scores]
        else:
            return [PRMResult(scores=scores, aggregation_method=aggregation_method).score for scores in all_scores]

In [5]:
user_message = """\
Hello! What is 1+2+3+...+100=?
"""

prompt = [
    {"role": "user", "content": user_message},
]
# response = await generate_response(prompt, max_tokens=256, return_raw=True)
response = await complete_response(user_message, max_tokens=256, return_raw=True)
logger.info(response)

/tmp/ipykernel_1349155/3145129444.py:9: RuntimeWarning: coroutine 'complete_response' was never awaited
  response = await complete_response(user_message, max_tokens=256, return_raw=True)
2025-06-16 00:25:01,081 | INFO | Task-10 | 3145129444:<module>:10 - Completion(id=None, choices=None, created=None, model=None, object='error', system_fingerprint=None, usage=None, message='The model does not support Completions API', type='BadRequestError', param=None, code=400)
